## Proyecto 3. Entrega 3
### LSTM
Diego Duarte 22075 - José Marchena 22398 - Andrés Kou 22305 - Esteban Zambrano 22119

In [1]:
import pandas as pd
import LSTM
import numpy as np

# Cargar el archivo CSV
csv_path = "../DataFinal.csv"
df = pd.read_csv(csv_path)

In [2]:
n_lags = 3  # Usaremos los 3 años previos
X, y = LSTM.preparar_series_lstm(df, n_lags)

# Revisar las dimensiones de X e y
print(X.shape)  # Esto debería ser (n_samples, 3, n_features)
print(y.shape)  




(114, 3, 20)
(114,)


In [3]:
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y prueba (sin mezclar las secuencias)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Ver las formas de los datos
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(91, 3, 20) (23, 3, 20)
(91,) (23,)


In [4]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# Normalizar X_train, X_test, y_train, y_test
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1]))

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Convertir los datos normalizados a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).view(-1, X_train.shape[1], X_train.shape[2])
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).view(-1, X_test.shape[1], X_test.shape[2])
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Crear dataloaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [9]:
# Definir parámetros
input_size = X_train.shape[2]  # El número de características (columnas de VI)
hidden_size = 64  # Puedes ajustar este valor
num_layers = 2  # Puedes ajustar este valor

# Inicializar el modelo
model = LSTM.LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

# Definir la función de pérdida (MSE) y el optimizador (Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
num_epochs = 50  # Número de épocas
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


for epoch in range(num_epochs):
    model.train()  # Establecer el modelo en modo entrenamiento
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        # Enviar los datos a GPU si está disponible
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reiniciar los gradientes de los optimizadores
        optimizer.zero_grad()

        # Hacer una predicción
        outputs = model(inputs)

        # Calcular la pérdida
        loss = criterion(outputs, labels)
        
        # Propagar hacia atrás (backpropagation)
        loss.backward()
        
        # Actualizar los parámetros
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 1.0150
Epoch [2/50], Loss: 0.9617
Epoch [3/50], Loss: 0.9063
Epoch [4/50], Loss: 0.8397
Epoch [5/50], Loss: 0.7556
Epoch [6/50], Loss: 0.6510
Epoch [7/50], Loss: 0.5288
Epoch [8/50], Loss: 0.4011
Epoch [9/50], Loss: 0.2891
Epoch [10/50], Loss: 0.2132
Epoch [11/50], Loss: 0.1682
Epoch [12/50], Loss: 0.1229
Epoch [13/50], Loss: 0.0745
Epoch [14/50], Loss: 0.0454
Epoch [15/50], Loss: 0.0383
Epoch [16/50], Loss: 0.0348
Epoch [17/50], Loss: 0.0272
Epoch [18/50], Loss: 0.0222
Epoch [19/50], Loss: 0.0204
Epoch [20/50], Loss: 0.0175
Epoch [21/50], Loss: 0.0145
Epoch [22/50], Loss: 0.0133
Epoch [23/50], Loss: 0.0129
Epoch [24/50], Loss: 0.0120
Epoch [25/50], Loss: 0.0109
Epoch [26/50], Loss: 0.0101
Epoch [27/50], Loss: 0.0095
Epoch [28/50], Loss: 0.0089
Epoch [29/50], Loss: 0.0083
Epoch [30/50], Loss: 0.0080
Epoch [31/50], Loss: 0.0079
Epoch [32/50], Loss: 0.0077
Epoch [33/50], Loss: 0.0075
Epoch [34/50], Loss: 0.0073
Epoch [35/50], Loss: 0.0070
Epoch [36/50], Loss: 0.0068
E

In [11]:
model.eval()  # Establecer el modelo en modo evaluación
with torch.no_grad():  # No necesitamos calcular los gradientes durante la evaluación
    y_pred = []
    y_true = []
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Hacer predicción
        outputs = model(inputs)
        
        # Almacenar las predicciones y los valores verdaderos
        y_pred.append(outputs.cpu().numpy())
        y_true.append(labels.cpu().numpy())
    
    y_pred = np.concatenate(y_pred, axis=0)
    y_true = np.concatenate(y_true, axis=0)

# Comparar las predicciones con los valores verdaderos
print(f"Predicciones: {y_pred[:5]}")
print(f"Valores reales: {y_true[:5]}")


Predicciones: [[ 0.56338555]
 [-0.19687498]
 [-0.7413362 ]
 [-1.0071175 ]
 [-1.0826098 ]]
Valores reales: [[ 0.35792658]
 [-0.29539424]
 [-0.8298666 ]
 [-1.041411  ]
 [-1.0556492 ]]
